### Objetivo general del análisis

El objetivo principal de este proyecto es utilizar las herramientas que ofrece la ciencia de datos provistas para:

- Analizar como se establece la oferta de vehículos usados en Europa.
- Investigar y representar vínculos significativos entre variables como potencia, kilometraje, año y precio.
- Identificar tendencias del mercado y posibles segmentos de automóviles (clusterización).
- Elaborar modelos de predicción de precios utilizando métodos supervisados (regresión lineal, árboles de decisión, entre otros).
- Implementar principios de éticos y de gobernanza en la gestión del dataset y transmitir los descubrimientos de manera visual y entendible.


In [ ]:
# Importamos librerías
import pandas as pd
import numpy as np

# Ignorar warnings
import warnings
warnings.filterwarnings('ignore')


# Cargamos el CSV
ruta_csv = "data/raw_vehicle_dataset.csv"
df = pd.read_csv(ruta_csv)


In [ ]:
# Damos un primer vistazo al dataset
print("Dimensiones del dataset:", df.shape)
print("\nTipos de datos:")
print(df.dtypes)
print("\nPrimeras filas:")
display(df.head())

In [ ]:
# Vemos la información general del dataset
print("\nInformación general:")
df.info()


In [ ]:
# Observamos los valores faltantes por columna
print("\nValores faltantes por columna:")
print(df.isnull().sum())

# Observamos el porcentaje de valores faltantes
print("\nPorcentaje de valores faltantes:")
print((df.isnull().sum() / len(df) * 100).round(2))


### 2.2 Limpieza y curación de datos

El dataset presenta 3.552.912 registros y 16 columnas, con múltiples variables numéricas y categóricas. La mayoría son datos estructurados, aunque algunas columnas presentan una cantidad significativa de valores faltantes.

In [ ]:
# Eliminamos columnas con gran cantidad de valores nulos
cols_a_eliminar = ["color_slug", "stk_year"]
df.drop(columns=cols_a_eliminar, inplace=True)


In [ ]:
# Convertimos fechas a datetime
df["date_created"] = pd.to_datetime(df["date_created"], errors="coerce")
df["date_last_seen"] = pd.to_datetime(df["date_last_seen"], errors="coerce")

# Convertimos columnas float a enteros donde corresponde (manteniendo nulos)
df["door_count"] = df["door_count"].astype("Int64")
df["seat_count"] = df["seat_count"].astype("Int64")


In [ ]:
# Normalizamos el texto
for col in ["maker", "model", "transmission", "fuel_type", "body_type"]:
    df[col] = df[col].astype(str).str.strip().str.lower().replace("nan", np.nan)


In [ ]:
# Eliminamos duplicados antes de imputar para no procesar datos redundantes
df.drop_duplicates(inplace=True)

# Eliminamos filas sin marca o modelo (no pueden ser interpretadas)
df.dropna(subset=["maker", "model"], inplace=True)


In [ ]:
# Imputamos con mediana
df["mileage"].fillna(df["mileage"].median(), inplace=True)
df["manufacture_year"].fillna(df["manufacture_year"].median(), inplace=True)
df["engine_power"].fillna(df["engine_power"].median(), inplace=True)
df["engine_displacement"].fillna(df["engine_displacement"].median(), inplace=True)


In [ ]:
# Imputamos con moda
df["transmission"].fillna(df["transmission"].mode()[0], inplace=True)
df["fuel_type"].fillna(df["fuel_type"].mode()[0], inplace=True)
df["door_count"].fillna(df["door_count"].mode()[0], inplace=True)
df["seat_count"].fillna(df["seat_count"].mode()[0], inplace=True)


In [ ]:
# Imputamos body_type como categoría "unknown"
df["body_type"].fillna("unknown", inplace=True)


### Verificación final del dataset tras la limpieza

- Se eliminaron las columnas con gran cantidad de valores faltantes y bajo valor informativo (`color_slug`, `stk_year`).
- La columna `body_type`, aunque presentaba una cantidad considerable de datos faltantes, fue conservada por su posible relevancia económica y se imputó con la categoría `"unknown"` para preservar la estructura sin inducir sesgos.
- Las variables numéricas `engine_power` y `engine_displacement`, que también presentaban una proporción significativa de valores faltantes, fueron imputadas mediante su mediana, con el objetivo de mantenerlas disponibles para el análisis predictivo sin introducir distorsiones excesivas.
- Se convirtieron correctamente las variables de fecha (`date_created`, `date_last_seen`) al tipo `datetime`.
- Se ajustaron los tipos de datos de `door_count` y `seat_count` a enteros con soporte para nulos (`Int64`).
- Se imputaron valores faltantes en variables clave usando la mediana para variables numéricas y la moda para variables categóricas o discretas, según correspondiera.
- Se estandarizaron los formatos de texto para las variables categóricas (`maker`, `model`, `transmission`, `fuel_type`).
- Se eliminaron registros duplicados completos.
- Se eliminaron registros sin información de `maker` o `model`, ya que son esenciales para el análisis.

> ### Nota desde el futuro:
`body_type` nos generará problemas debido a que no solo tiene muchos valores faltantes, sino que dentro de los valores presentes la mayoría es constituida por "other" por lo que deja de ofrecer información objetiva y será eliminada del dataset en una celda futura.



In [ ]:
print(" Dimensiones finales del dataset:", df.shape)

# Verifcamos los tipos de datos
print("\n Tipos de datos finales:")
print(df.dtypes)

# Verificamos de valores nulos restantes
print("\n Valores faltantes por columna (deberían ser 0 o mínimos):")
print(df.isnull().sum())

# Verificamos duplicados
print("\n Registros duplicados restantes:")
print(df.duplicated().sum())

# Damos una vista rápida final
print("\n Primeras filas del DataFrame limpio:")
display(df.head())


## 3. Análisis exploratorio y visualización

En esta sección se realiza un análisis exploratorio de datos (EDA), con el objetivo de identificar patrones, distribuciones, relaciones y posibles anomalías en el dataset. La exploración se apoya en técnicas de visualización para facilitar la interpretación de los datos y sentar las bases del modelado posterior.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

# Seteamos la configuración estética general para los gráficos con seaborn y matplotlib
sns.set(style="whitegrid")
plt.rcParams["figure.figsize"] = (10, 5)


In [ ]:
# Graficamos las distribuciones para las variables numéricas del dataset completo
numericas = ["price_eur", "mileage", "engine_power", "engine_displacement", "manufacture_year"]

for col in numericas:
    plt.figure()
    sns.histplot(data=df, x=col, kde=True, bins=50)
    plt.title(f"Distribución de {col}")
    plt.xlabel(col)
    plt.ylabel("Frecuencia")
    plt.tight_layout()
    plt.show()


###  Filtrado de outliers extremos para visualización

Se identificaron valores atípicos en variables como `price_eur`, `mileage`, `engine_displacement` y `manufacture_year` que distorsionan las visualizaciones. Por ello, se construye una copia filtrada del dataset (`df_vis`) para generar gráficos más representativos del mercado real.


In [ ]:
# Creamos una copia del dataframe original para visualización (sin modificar df)
df_vis = df.copy()

# Filtramos precios extremos fuera del rango realista para autos usados (ej: >150.000 EUR)
df_vis = df_vis[df_vis["price_eur"].between(500, 150000)]

# Limpiamos valores de kilometraje absurdamente altos (> 1 millón de km)
df_vis = df_vis[df_vis["mileage"] < 1_000_000]

# Eliminamos autos con cilindrada superior a 10.000 cc (poco probable en autos comerciales)
df_vis = df_vis[df_vis["engine_displacement"] < 10000]

# Eliminamos autos anteriores a 1980 (pocos registros, posible error de carga o antigüedades)
df_vis = df_vis[df_vis["manufacture_year"] >= 1980]

# Mostramos cuántos registros quedaron para las visualizaciones
print("Cantidad de registros tras filtrado para visualización:", df_vis.shape)


In [ ]:
# Graficamos las distribuciones univariadas tras el filtrado (df_vis) para un análisis más prolijo
numericas = ["price_eur", "mileage", "engine_power", "engine_displacement", "manufacture_year"]

for col in numericas:
    plt.figure()
    sns.histplot(data=df_vis, x=col, kde=True, bins=50)
    plt.title(f"Distribución de {col}")
    plt.xlabel(col)
    plt.ylabel("Frecuencia")
    plt.tight_layout()
    plt.show()


> Los gráficos muestran la distribución individual de las principales variables numéricas del dataset. Se observa una fuerte asimetría a la derecha en `price_eur`, `mileage` y `engine_power`, indicando que la mayoría de los vehículos se concentran en valores bajos, con algunos casos extremos que elevan los máximos. Por su parte, `manufacture_year` muestra una clara concentración de vehículos producidos entre 2005 y 2016, con picos que podrían asociarse a políticas de renovación o tendencias de ventas


In [ ]:
# Definimos una función para graficar variables categóricas ordenadas por frecuencia
def plot_categorical(col, top_n=None):
    plt.figure()
    order = df[col].value_counts().head(top_n).index if top_n else df[col].value_counts().index
    sns.countplot(data=df, x=col, order=order)
    plt.title(f"Distribución de {col}")
    plt.xticks(rotation=45)
    plt.xlabel(col)
    plt.ylabel("Cantidad")
    plt.tight_layout()
    plt.show()


In [ ]:
# Creamos una lista con las variables categóricas
categoricas = ["fuel_type", "transmission", "door_count", "seat_count", "body_type"]
for col in categoricas:
    plot_categorical(col)

# Creamos un gráfico especial para las top 10 marcas más frecuentes
plot_categorical("maker", top_n=10)


> Se observa que la gran mayoría de los vehículos utilizan gasolina y son manuales, lo que refleja un perfil técnico dominante en el mercado. En cuanto al número de puertas y asientos, 5 puertas y 5 asientos es lo más frecuente, lo que sugiere una prevalencia de autos familiares o utilitarios. En la variable `body_type`, la categoría other domina por sobre el resto, seguida de un alto porcentaje de valores unknown, lo que nos lleva a tomar la decisión de descartar esta variable del análisis. Finalmente, en cuanto a las marcas, Skoda y Volkswage lideran ampliamente el mercado.


### Eliminación de la variable `body_type`

Si bien inicialmente se imputaron los valores faltantes de `body_type` con la categoría `"unknown"`, el análisis posterior mostró que más del 90% de los registros se concentraban en las categorías `"other"` y `"unknown"`, con muy poca representatividad de los tipos reales de carrocería. Esto se observa claramente en el gráfico de barras provisto. Debido a la escasa utilidad explicativa y el riesgo de introducir ruido o sesgos, se decidió eliminar esta variable del análisis y modelado final.


In [ ]:
# Eliminamos la variable body_type por baja utilidad analítica
df.drop(columns=["body_type"], inplace=True)

In [ ]:
# Exportamos la versión final del dataset limpio
df.to_csv("data/clean_vehicle_dataset.csv", index=False)


## 3.2 Relaciones entre variables

En esta sección se analiza la relación entre variables clave del dataset. Se utilizan gráficos de dispersión para visualizar relaciones entre variables numéricas, boxplots para comparar variables numéricas por categoría, y un mapa de calor para detectar correlaciones lineales entre variables cuantitativas.


In [ ]:
# Graficamos las relaciones entre variables numéricas clave
sns.pairplot(data=df_vis, vars=["price_eur", "mileage", "engine_power", "engine_displacement"], plot_kws={"alpha":0.3})
plt.suptitle("Relaciones entre variables numéricas", y=1.02)
plt.show()


> Este gráfico permite observar la relación entre variables clave como precio, kilometraje, potencia y cilindrada. Se destaca una relación inversa entre `price_eur` y `mileage`, así como una correlación positiva entre `engine_power` y `engine_displacement`.

In [ ]:
# Graficamos precio vs kilometraje
sns.scatterplot(data=df_vis, x="mileage", y="price_eur", alpha=0.3)
plt.title("Precio vs Kilometraje")
plt.show()


> Se observa una clara relación negativa: a mayor kilometraje, menor es el precio del vehículo. Esta tendencia refuerza la idea de que el desgaste del auto es un determinante clave del valor de reventa.

In [ ]:
# Graficamos precio según tipo de combustible
plt.figure()
sns.boxplot(data=df_vis, x="fuel_type", y="price_eur")
plt.title("Precio según tipo de combustible")
plt.xticks(rotation=45)
plt.show()

# Graficamos precio según tipo de transmisión
plt.figure()
sns.boxplot(data=df_vis, x="transmission", y="price_eur")
plt.title("Precio según tipo de transmisión")
plt.show()


> Los vehículos diésel y gasolina tienen precios similares, aunque los diésel muestran una mayor dispersión. El resto de los combustibles alternativos (lpg, cng, electric) presentan menor representación y precios más bajos en promedio.
>
> Los autos con transmisión automática tienden a tener precios más altos y mayor dispersión que los manuales. Esto puede deberse a que los automáticos suelen asociarse a modelos más nuevos o de gama más alta.

In [ ]:
# Graficamos el mapa de correlación entre numéricas
corr = df_vis[["price_eur", "mileage", "engine_power", "engine_displacement", "manufacture_year"]].corr()

plt.figure(figsize=(8,6))
sns.heatmap(corr, annot=True, cmap="coolwarm", fmt=".2f")
plt.title("Matriz de correlación entre variables numéricas")
plt.show()


### 3.2 Análisis e interpretación de relaciones entre variables

A partir de los gráficos generados, se destacan los siguientes patrones:

- **Precio vs. Kilometraje**: se observa una clara relación negativa. A mayor kilometraje, menor precio. Esto es coherente con la depreciación natural del valor del vehículo por uso.

- **Precio vs. Potencia (`engine_power`)**: la correlación positiva indica que los autos más potentes tienden a tener mayor precio. La relación es visible pero no lineal.
  
- **Potencia vs. Cilindrada**: la correlación es muy alta (~0.73), lo que indica una fuerte relación entre ambas variables técnicas del motor.
  
- **Boxplot por tipo de combustible**: los vehículos diésel y gasolina dominan el mercado. El precio es más alto en general en ambos, mientras que GNC, GLP y eléctricos están subrepresentados (posiblemente outliers o mal cargados).

- **Boxplot por tipo de transmisión**: los vehículos con transmisión automática (`auto`) tienen, en promedio, un precio significativamente más alto que los manuales (`man`). Esto puede deberse a su asociación con gamas más altas o vehículos de lujo, además de un mercado más reducido.


- **Año de fabricación** tiene correlación positiva con el precio: autos más nuevos → precios más altos, como era de esperarse.

- **Mapa de correlaciones**:
  - `price_eur` tiene correlación:
    - Negativa con `mileage` (-0.44)
    - Positiva con `engine_power` (0.57)
    - Positiva con `manufacture_year` (0.43)
  - Cabe destacar que las correlaciones no son perfectas, lo que indica que el precio está influido por múltiples factores combinados.


## 3.3 Comparaciones por categorías

Se comparan precios y otras variables según distintas categorías relevantes. Estas comparaciones permiten identificar diferencias en el valor del vehículo según características técnicas y comerciales. Se utilizan principalmente boxplots y gráficos de barras agrupados.


In [ ]:
# Graficamos el precio según cantidad de puertas
plt.figure()
sns.boxplot(data=df_vis, x="door_count", y="price_eur")
plt.title("Precio según cantidad de puertas")
plt.xlabel("Cantidad de puertas")
plt.ylabel("Precio (€)")
plt.show()


In [ ]:
# Calculamos el precio promedio por marca y ordenar de forma descendente
top_makers = df_vis["maker"].value_counts().head(10).index
avg_price_by_maker = df_vis[df_vis["maker"].isin(top_makers)].groupby("maker")["price_eur"].mean().sort_values(ascending=False)


In [ ]:
# Graficamos el precio promedio por marca (top 10)
plt.figure()
sns.barplot(x=avg_price_by_maker.index, y=avg_price_by_maker.values)
plt.title("Precio promedio por marca (top 10)")
plt.ylabel("Precio promedio (€)")
plt.xticks(rotation=45)
plt.show()


In [ ]:
# Agrupamos por marca y transmisión (solo top 5 marcas para que no sea ilegible)
top_5_makers = df_vis["maker"].value_counts().head(5).index
grouped = df_vis[df_vis["maker"].isin(top_5_makers)].groupby(["maker", "transmission"])["price_eur"].mean().reset_index()

In [ ]:
# Graficamos el precio promedio por marca y tipo de transmisión
plt.figure()
sns.barplot(data=grouped, x="maker", y="price_eur", hue="transmission")
plt.title("Precio promedio por marca y tipo de transmisión")
plt.ylabel("Precio (€)")
plt.show()


### 3.3 Comparaciones por categorías

Se analizaron los precios de los vehículos en función de distintas variables categóricas y discretas. A continuación se presentan los hallazgos más relevantes:

- **Precio según cantidad de puertas (`door_count`)**: el valor más frecuente es de 3 a 5 puertas, que concentra la mayoría del parque automotor. Hay algunos valores extremos atípicos (22, 45, etc.) que probablemente correspondan a errores de carga o a otro tipo de vehículos (ej. colectivos, buses). El precio no varía significativamente entre 3, 4 y 5 puertas.

- **Precio según cantidad de asientos (`seat_count`)**: la mayoría de los autos tienen entre 4 y 7 asientos. A mayor número de asientos, especialmente en configuraciones de 8 a 10, se observan precios más altos, lo cual puede relacionarse con vehículos familiares, utilitarios o de transporte comercial.

- **Precio promedio por marca (`maker`)**: entre las 10 marcas más frecuentes, se destaca **Audi** con un precio promedio muy superior, lo cual refleja su posicionamiento premium. Le siguen Volkswagen, Seat y Ford. Skoda, Renault y Peugeot presentan precios más accesibles.

- **Precio promedio por marca y transmisión**: en todas las marcas analizadas (top 5), los vehículos con transmisión automática son consistentemente más caros que los manuales. Este patrón es más marcado en marcas como Audi y Volkswagen.


### 3.4 Visualizaciones interactivas con Plotly

A continuación se presentan visualizaciones interactivas diseñadas para facilitar la exploración dinámica de las relaciones entre variables. Estas herramientas permiten detectar patrones relevantes, outliers, y diferencias entre categorías de forma visual e intuitiva.


In [ ]:
import plotly.express as px
import plotly.io as pio

# Específicamos el tipo de renderización para vscode
pio.renderers.default = "vscode"

# Si se quiere para google colab o jupyter notebook cambiar por "colab" o "notebook" respectivamente

In [ ]:
# Creamos muestra aleatoria de 5000 vehículos para evitar que el gráfico colapse por exceso de puntos
df_sample = df_vis.sample(5000, random_state=42)

In [ ]:

# Creamos un gráfico de dispersión interactivo entre precio y potencia del motor, coloreado por tipo de combustible
fig = px.scatter(
    df_sample,
    x="engine_power",
    y="price_eur",
    color="fuel_type",
    hover_data=["maker", "model", "mileage", "manufacture_year"],
    title="Precio vs Potencia del Motor (coloreado por tipo de combustible)",
    labels={"engine_power": "Potencia (kW)", "price_eur": "Precio (€)"},
    opacity=0.4
)

fig.show()


> Se observa que los autos con mayor potencia tienden a tener precios más altos, aunque con mucha dispersión. Los tipos de combustible predominantes son gasolina y diésel. Los autos eléctricos y a gas son escasos y se concentran en precios y potencias bajas.


In [ ]:
# Graficamos la distribución de precios por tipo de combustible
fig = px.box(
    df_vis[df_vis["fuel_type"].isin(["gasoline", "diesel"])],
    x="fuel_type",
    y="price_eur",
    points="outliers",
    title="Distribución de precios por tipo de combustible (solo gasolina y diésel)",
    labels={"fuel_type": "Tipo de combustible", "price_eur": "Precio (€)"}
)
fig.show()


> Gasolina y diésel presentan distribuciones de precios similares, aunque los vehículos diésel muestran más outliers hacia valores altos. Esto podría reflejar que algunos modelos diésel están orientados al segmento comercial o de alta gama.


In [ ]:
# Graficamos la distribución de precios por tipo de transmisión
fig = px.box(
    df_vis,
    x="transmission",
    y="price_eur",
    points="outliers",
    title="Distribución de precios por tipo de transmisión",
    labels={"transmission": "Transmisión", "price_eur": "Precio (€)"}
)
fig.show()


> Los autos con transmisión automática tienen un precio mediano mayor que los manuales. Además, presentan una mayor dispersión de precios, lo que sugiere que incluyen tanto autos de lujo como recientes.


In [ ]:
# Calculamos el precio promedio por marca (top 10)
top_makers = df_vis["maker"].value_counts().head(10).index
avg_price = df_vis[df_vis["maker"].isin(top_makers)].groupby("maker")["price_eur"].mean().reset_index()

In [ ]:
# Creamos un gráfico de barras para el precio promedio por marca (top 10)
fig = px.bar(
    avg_price.sort_values("price_eur", ascending=False),
    x="maker",
    y="price_eur",
    title="Precio promedio por marca (top 10)",
    labels={"price_eur": "Precio promedio (€)", "maker": "Marca"}
)
fig.show()


> Audi lidera claramente en precio promedio, lo cual es coherente con su posicionamiento como marca premium. Volkswagen y Seat también tienen valores elevados. Las marcas con menor precio promedio son Peugeot, Renault y Skoda, más asociadas a autos de gama media o económica.

In [ ]:
# Calculamos una nueva variable "car_age" que es la antigüedad del vehículo
df_vis["car_age"] = df_vis["date_created"].dt.year - df_vis["manufacture_year"]


In [ ]:
# Generamos un gráfico de dispersión interactivo entre precio y antigüedad del vehículo, coloreado por tipo de transmisión
fig = px.scatter(
    df_vis,
    x="car_age",
    y="price_eur",
    color="transmission",
    hover_data=["maker", "model", "engine_power"],
    title="Precio vs Antigüedad del vehículo (coloreado por tipo de transmisión)",
    labels={"car_age": "Años de antigüedad", "price_eur": "Precio (€)"},
    opacity=0.4
)

fig.show()


> Se confirma una relación inversa entre precio y antigüedad: los autos más nuevos valen más. Los automáticos predominan en los vehículos recientes, mientras que los manuales son mayoría en los modelos más antiguos.


### Visualizaciones interactivas complementarias

Además de los gráficos analíticos presentados anteriormente, se incorporan a continuación tres visualizaciones sencillas e intuitivas, diseñadas para facilitar una comprensión rápida del perfil general del mercado. Estas representaciones permiten responder preguntas frecuentes de cualquier usuario, como qué tipo de combustible es más común, cuántos autos son manuales o automáticos, y qué tan antiguos son los vehículos publicados.


In [ ]:
# Visualizamos la proporción de autos por tipo de combustible usando un gráfico de torta
fig = px.pie(
    df_vis,
    names="fuel_type",
    title="Distribución de vehículos por tipo de combustible",
    hole=0.4
)
fig.show()


> Se observa que el 81 % de los vehículos utilizan gasolina, seguido por un 17 % que utiliza diésel. Los tipos eléctricos, a gas o alternativos representan menos del 1 %. La predominancia de combustibles tradicionales es clara, lo cual es relevante al interpretar los precios o segmentar el mercado.

In [ ]:
# Visualizamos la proporción de autos por tipo de transmisión (manual vs automática)
fig = px.pie(
    df_vis,
    names="transmission",
    title="Distribución de vehículos por tipo de transmisión",
    hole=0.4
)
fig.show()


> Se observa que más del 81 % de los autos del dataset tienen transmisión manual, frente a un 18 % con transmisión automática. Esto indica una fuerte inclinación del mercado hacia vehículos de menor costo, dado que la transmisión automática suele asociarse a gamas más altas o a preferencias específicas

In [ ]:
# Mostramos la distribución de autos según su año de fabricación
fig = px.histogram(
    df_vis,
    x="manufacture_year",
    nbins=10,
    title="Distribución de vehículos según año de fabricación",
    labels={"manufacture_year": "Año de fabricación"}
)
fig.show()


> El histograma muestra una concentración creciente de autos fabricados entre 2000 y 2015, con un pico alrededor de 2005–2010. Esto sugiere que la mayoría de los vehículos listados son usados pero no excesivamente antiguos, lo que tiene implicancias tanto para su precio como para su estado general esperado.

## 4. Segmentación de mercado mediante Clustering (K-Means)

En esta sección se aplica el algoritmo de K-Means para identificar grupos de vehículos con características similares. Esta técnica de aprendizaje no supervisado permite segmentar el mercado sin necesidad de variables objetivo.

Se trabaja con variables numéricas seleccionadas por su relevancia comercial y su relación con el precio:

- `engine_power` (potencia del motor)
- `mileage` (kilometraje)
- `car_age` (antigüedad del vehículo)

Estas variables fueron previamente limpiadas y filtradas, y se normalizan antes de aplicar el algoritmo.


In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

# Usamos una muestra para reducir carga computacional
df_cluster = df_vis[["engine_power", "mileage", "date_created", "manufacture_year"]].dropna().copy()
df_cluster = df_cluster.sample(5000, random_state=42)

# Creamos la variable de antigüedad
df_cluster["car_age"] = df_cluster["date_created"].dt.year - df_cluster["manufacture_year"]

# Seleccionamos variables numéricas para el clustering
X = df_cluster[["engine_power", "mileage", "car_age"]]

# Normalizamos
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)


In [ ]:
# Aplicamos KMeans con 4 clusters (puede ajustarse según el elbow method)
kmeans = KMeans(n_clusters=4, random_state=42, n_init=10)
df_cluster["cluster"] = kmeans.fit_predict(X_scaled)


In [ ]:
# Aplicamos PCA para reducción a 2D
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_scaled)

df_cluster["PCA1"] = X_pca[:, 0]
df_cluster["PCA2"] = X_pca[:, 1]




In [ ]:
import plotly.express as px

# Creamos un gráfico interactivo
fig = px.scatter(
    df_cluster, x="PCA1", y="PCA2", color=df_cluster["cluster"].astype(str),
    hover_data=["engine_power", "mileage", "car_age"],
    title="Clusters de vehículos según características técnicas (K-Means + PCA)",
    labels={"cluster": "Cluster"}
)
fig.show()

> El algoritmo K-Means identificó 4 grupos distintos de vehículos según potencia, kilometraje y antigüedad. La proyección en 2D (mediante PCA) muestra que los clusters están bien diferenciados, aunque con algunas zonas de superposición.
>
> - Algunos clusters agrupan vehículos con **alta potencia y bajo kilometraje**, posiblemente autos nuevos de gama alta.
> - Otros se concentran en autos con **alta antigüedad y muchos kilómetros**, más depreciados.
> - El análisis de estos grupos puede ayudar a definir perfiles de mercado y estrategias comerciales diferenciadas.
>
> Se realizará un mejor análisis con un gráfico de barras a continuación.


In [ ]:
# Creamos un resumen de promedios por cluster y guardarlo para análisis posterior
cluster_summary = df_cluster.groupby("cluster")[["engine_power", "mileage", "car_age"]].mean().reset_index()
cluster_summary["cluster"] = cluster_summary["cluster"].astype(int)


In [ ]:
# Normalizamos los promedios por variable antes de graficar (min-max scaling por columna)
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
cluster_scaled = cluster_summary.copy()
cluster_scaled[["engine_power", "mileage", "car_age"]] = scaler.fit_transform(
    cluster_scaled[["engine_power", "mileage", "car_age"]]
)

# Reestructuramos para visualización comparativa entre clusters
cluster_scaled_melted = cluster_scaled.melt(
    id_vars="cluster",
    var_name="variable",
    value_name="valor_normalizado"
)


In [ ]:
# Creamos un gráfico de barras para la comparación de variables técnicas normalizadas por cluster
plt.figure(figsize=(8,5))
ax = sns.barplot(data=cluster_scaled_melted, x="variable", y="valor_normalizado", hue="cluster")
plt.title("Comparación relativa por variable dentro de cada cluster (valores normalizados)")
plt.ylabel("Valor normalizado (0–1)")

# Agregamos etiquetas numéricas para confirmar presencia de valores ocultos
for container in ax.containers:
    ax.bar_label(container, fmt="%.2f")

plt.tight_layout()
plt.show()


> **Interpretación de clusters:**
>
> Este gráfico permite observar de forma relativa cómo se comportan los distintos clusters en cada variable clave. Al normalizar las variables entre 0 y 1, podemos comparar su magnitud independientemente de sus unidades.
> Es importante aclarar que los valores 0.00 y 1.00 son posiciones relativas dentro de la muestra, no valores brutos.
>
>  - **Cluster 0** se destaca con valores máximos de `engine_power` y bajos de `car_age`, lo que indica autos muy potentes y recientes. Además, tienen un `mileage` medio-bajo. Esto sugiere que podrían tratarse de vehículos deportivos o de alta gama.
>
> - **Cluster 1** tiene valores intermedios en todas las variables, pero se observa potencia baja, `engine_power` es bajo, sobretodo si lo compramos con el cluster anterior. Representa un segmento medio, probablemente autos usados estables, pero ni nuevos ni potentes.
>
> - **Cluster 2** presenta valores máximos en `mileage` y `car_age`, y valores mínimos en `engine_power`. Es el grupo de vehículos más antiguos y desgastados, con baja potencia. Probablemente autos económicos y en su etapa final de vida útil.
>
> - **Cluster 3** tiene muy baja antigüedad y kilometraje, pero `engine_power` también bajo-moderado. Se interpreta como autos nuevos, pero económicos, posiblemente compactos o de ciudad.


## 5. Modelado predictivo del precio

En esta sección se entrena un modelo supervisado para predecir el precio (`price_eur`) de un vehículo a partir de sus características técnicas.

Se comparan dos modelos:

- Regresión lineal (modelo base, interpretable)
- Árbol de decisión (modelo no lineal, con capacidad para capturar relaciones más complejas)

Se utilizarán las siguientes variables predictoras:

- `engine_power`
- `mileage`
- `car_age`
- `transmission`
- `fuel_type`


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

# Creamos una copia del dataset y eliminamos nulos
df_model = df_vis[["price_eur", "engine_power", "mileage", "manufacture_year", "date_created", "transmission", "fuel_type"]].dropna().copy()

# Especificamos la variable target
y = df_model["price_eur"]

# Especificamos la variable derivada: antigüedad
df_model["car_age"] = df_model["date_created"].dt.year - df_model["manufacture_year"]

# Especificamos las variables predictoras
X = df_model[["engine_power", "mileage", "car_age", "transmission", "fuel_type"]]

# Creamos la división entrenamiento-test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
# Definimos las columnas numéricas y categóricas del modelo
num_cols = ["engine_power", "mileage", "car_age"]
cat_cols = ["transmission", "fuel_type"]


In [ ]:
# Armamos el preprocesador que normaliza las numéricas y codifica las categóricas
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

preprocessor = ColumnTransformer([
    ("num", StandardScaler(), num_cols),
    ("cat", OneHotEncoder(handle_unknown="ignore"), cat_cols)
])


In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

# Definimos la función para evaluar cualquier modelo con métricas estándar
def evaluar(modelo, X_test, y_test, nombre="Modelo"):
    pred = modelo.predict(X_test)
    print(f"{nombre}")
    print(" MAE :", round(mean_absolute_error(y_test, pred), 2))
    print(" RMSE:", round(np.sqrt(mean_squared_error(y_test, pred)), 2))
    print(" R²  :", round(r2_score(y_test, pred), 3))
    print("-" * 30)


In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBRegressor

# Definimos 3 modelos base que se usarán dentro de pipelines
modelos = {
    "Regresión Lineal": LinearRegression(),
    "Árbol de Decisión": DecisionTreeRegressor(max_depth=6, random_state=42),
    "XGBoost": XGBRegressor(
        n_estimators=100,
        max_depth=5,
        learning_rate=0.1,
        objective="reg:squarederror",
        random_state=42
    )
}


In [ ]:
# Entrenamos cada modelo dentro de un pipeline completo con preprocesamiento
from sklearn.pipeline import Pipeline

for nombre, modelo_base in modelos.items():
    modelo = Pipeline([
        ("preprocessing", preprocessor),
        ("regressor", modelo_base)
    ])
    modelo.fit(X_train, y_train)
    evaluar(modelo, X_test, y_test, nombre)


In [ ]:
# Creamos una lista vacía para guardar los resultados
resultados = []

# Entrenamos nuevamente y guardamos métricas para cada modelo
for nombre, modelo_base in modelos.items():
    modelo = Pipeline([
        ("preprocessing", preprocessor),
        ("regressor", modelo_base)
    ])
    modelo.fit(X_train, y_train)
    pred = modelo.predict(X_test)

    mae = mean_absolute_error(y_test, pred)
    rmse = np.sqrt(mean_squared_error(y_test, pred))
    r2 = r2_score(y_test, pred)

    resultados.append({
        "Modelo": nombre,
        "MAE": round(mae, 2),
        "RMSE": round(rmse, 2),
        "R²": round(r2, 3)
    })


In [ ]:
# Convertimos a DataFrame y mostramos resultados
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

df_resultados = pd.DataFrame(resultados)
display(df_resultados)

# Creamos un gráfico comparativo (R²)
plt.figure(figsize=(7,4))
sns.barplot(data=df_resultados, x="Modelo", y="R²")
plt.title("Comparación de desempeño (R²)")
plt.ylim(0, 1)
plt.tight_layout()
plt.show()


### Conclusión del modelado predictivo

| Modelo              | MAE (↓)   | RMSE (↓)  | R² (↑)   |
|---------------------|-----------|-----------|---------|
| Regresión Lineal    | ~5100 €   | ~8500 €   | 0.527    |
| Árbol de Decisión   | ~4200 €   | ~7300 €   | 0.654   |
| **XGBoost**         | **~3600 €** | **~6569 €** | **0.715** |

## Se compararon tres modelos de regresión para predecir el precio de vehículos usados a partir de características técnicas y categóricas:

- **Regresión Lineal**: modelo simple e interpretable con un (R² = 0.527). Sirve como punto de referencia, pero no captura relaciones no lineales ni interacciones entre variables.

- **Árbol de Decisión**: mejora sustancial en precisión (R² = 0.654), gracias a su capacidad para particionar el espacio de decisiones y adaptarse a relaciones complejas. Permite cierta interpretabilidad estructural.

- **XGBoost**: modelo de boosting que combina múltiples árboles. Fue el más preciso (R² = 0.715), con el menor MAE y RMSE. Captura de manera eficiente las relaciones no lineales, combinaciones de categorías, y variaciones del mercado.

---




## 5.1 Aplicación del modelo a datos reales (ejemplo)

Luego de evaluar los distintos modelos predictivos y seleccionar XGBoost como el más preciso, se procede a realizar un ejemplo de aplicación práctica. Esta etapa consiste en utilizar el modelo entrenado para estimar el precio de autos reales del dataset que cuentan con características completas (potencia, kilometraje, antigüedad, tipo de transmisión y combustible).

In [ ]:
# Crear pipeline XGBoost con preprocesamiento
xgb = Pipeline([
    ("preprocessing", preprocessor),
    ("regressor", XGBRegressor(
        n_estimators=100,
        max_depth=5,
        learning_rate=0.1,
        objective="reg:squarederror",
        random_state=42
    ))
])


In [ ]:
# Entrenamos el modelo con conjunto original
xgb.fit(X_train, y_train)  # ← asume que X_train e y_train ya están definidos

# Filtramos los autos válidos con datos completos
autos_validos = df_vis[["engine_power", "mileage", "manufacture_year", "date_created",
                        "transmission", "fuel_type", "maker", "model", "price_eur"]].dropna()

# Creamos la variable car_age
autos_validos["car_age"] = autos_validos["date_created"].dt.year - autos_validos["manufacture_year"]

# Seleccionamos los primeros 20 para mostrar
autos_validos = autos_validos.head(20).copy()

# Especificamos las variables predictoras
X_validos = autos_validos[["engine_power", "mileage", "car_age", "transmission", "fuel_type"]]

# Predecimos usando el pipeline entrenado
autos_validos["predicted_price"] = xgb.predict(X_validos)

# Comparamos precios
autos_validos[["maker", "model", "engine_power", "car_age", "transmission",
               "fuel_type", "price_eur", "predicted_price"]]

In [ ]:
# Graficamos el Precio real vs. Precio predicho
plt.figure(figsize=(8,5))
sns.scatterplot(data=autos_validos, x="price_eur", y="predicted_price")
plt.plot([0, autos_validos["price_eur"].max()], [0, autos_validos["price_eur"].max()], ls="--", c="gray")
plt.xlabel("Precio real (€)")
plt.ylabel("Precio predicho (€)")
plt.title("Comparación: Precio real vs. Precio estimado (XGBoost)")
plt.tight_layout()
plt.show()


In [ ]:
# Calculamos el error absoluto para cada predicción
autos_validos["error_absoluto"] = abs(autos_validos["price_eur"] - autos_validos["predicted_price"])

# Mostramos los 10 autos donde el modelo más se equivocó
autos_validos.sort_values("error_absoluto", ascending=False)[
    ["maker", "model", "engine_power", "car_age", "price_eur", "predicted_price", "error_absoluto"]
].head(10)


### Análisis del desempeño del modelo

- Podemos medir el desempeño desde línea diagonal que representa el caso ideal donde el precio estimado coincide perfectamente con el real.
- La mayoría de los puntos están cerca de esta diagonal, lo que refleja un buen desempeño general.
- Sin embargo, existen desvíos que se observan claramente en la tabla provista:
  - **Subestimaciones**: como el Ford Galaxy ~10.500 € reales vs ~6.200 € estimados, o el Skoda Octavia.
  - **Sobreestimaciones**: como el Suzuki Swift, con un precio real de ~7.400 €, pero una predicción superior a 14.000 €.

---
>
### Conclusión

El modelo XGBoost logra un desempeño sólido en la mayoría de los casos. Aunque algunos errores puntuales existen, especialmente en autos menos frecuentes o con características particulares, las predicciones son razonables y consistentes para un uso práctico. Esto valida su aplicación como herramienta de estimación automática de precios en plataformas de venta de autos usados.
